In [ ]:
#konlpy downloading. 
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 453 kB 56.9 MB/s 
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.6 LTS"
/etc/os-release:NAME="Ubuntu"
/etc/os-release:ID=ubuntu
/etc/os-release:ID_LIKE=debian
/e

In [ ]:
import pickle

from konlpy.tag import Mecab

import numpy as np
from itertools import permutations

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 여러가지 시도

In [ ]:
with open("/content/gdrive/MyDrive/NLP/test_hobbyit.p", "rb") as file:
  model=pickle.load(file)

In [ ]:
# .freq('단어') -> 확률 추출
model.unigrams.freq('.')

0.03746172880572211

In [ ]:
model.unigrams['박준']

138

In [ ]:
model['박준']

138

In [ ]:
model[['박준']]

FreqDist({'동': 51, '국': 23, '시인': 16, '기자': 10, '선수': 5, '플랫폼': 3, '은': 3, '을': 2, '탁': 2, '길': 2, ...})

In [ ]:
len(model[['박준']])

27

In [ ]:
model[['박준']]['을']

2

In [ ]:
model[['박준']].freq('을')

0.014492753623188406

In [ ]:
len(model[['킼']])

0

In [ ]:
if model[['킼']]:
  print('yes')

In [ ]:
np.array([3,2,7]).argmax()

2

In [ ]:
def softmax(x):

    y = np.exp(x - np.max(x))
    f_x = y / np.sum(y)
    return f_x

In [ ]:
softmax([3,2,7])

array([0.01786798, 0.00657326, 0.97555875])

In [ ]:
mecab = Mecab()
mecab.pos("예쁘")

[('예쁘', 'VA')]

In [ ]:
mecab.pos("예쁘다")

[('예쁘', 'VA'), ('다', 'EC')]

In [ ]:
combs = [("강의", "예쁘", "네트워크", "연결"), ('키', '킥', '의의')]

In [ ]:
list(permutations(combs[0]))

[('강의', '예쁘', '네트워크', '연결'),
 ('강의', '예쁘', '연결', '네트워크'),
 ('강의', '네트워크', '예쁘', '연결'),
 ('강의', '네트워크', '연결', '예쁘'),
 ('강의', '연결', '예쁘', '네트워크'),
 ('강의', '연결', '네트워크', '예쁘'),
 ('예쁘', '강의', '네트워크', '연결'),
 ('예쁘', '강의', '연결', '네트워크'),
 ('예쁘', '네트워크', '강의', '연결'),
 ('예쁘', '네트워크', '연결', '강의'),
 ('예쁘', '연결', '강의', '네트워크'),
 ('예쁘', '연결', '네트워크', '강의'),
 ('네트워크', '강의', '예쁘', '연결'),
 ('네트워크', '강의', '연결', '예쁘'),
 ('네트워크', '예쁘', '강의', '연결'),
 ('네트워크', '예쁘', '연결', '강의'),
 ('네트워크', '연결', '강의', '예쁘'),
 ('네트워크', '연결', '예쁘', '강의'),
 ('연결', '강의', '예쁘', '네트워크'),
 ('연결', '강의', '네트워크', '예쁘'),
 ('연결', '예쁘', '강의', '네트워크'),
 ('연결', '예쁘', '네트워크', '강의'),
 ('연결', '네트워크', '강의', '예쁘'),
 ('연결', '네트워크', '예쁘', '강의')]

In [ ]:
made = []
prob = []

for t in combs:
  per = list(permutations(t))
  for k in range(len(per)):
    input = per[k]
    n = len(input)
    sentence = ""
    pp = ['은', '는', '이', '가', '을', '를', '와', '과', '다', '고']
    flag = 1
    total_freq = model.unigrams.freq(input[0])
    for i in range(n-1):
      if model[[input[i]]]:
        if model[[input[i]]][input[i+1]]:
          sentence += input[i] + " "
          total_freq *= model[[input[i]]].freq(input[i+1])
        else:
          # 조사 활용
          pre_freq = []
          post_freq = []
          for p in pp:
            pre_freq.append(model[[input[i]]].freq(p))
            post_freq.append(model[[p]].freq(input[i+1]))
          con_freq = np.array(pre_freq) * np.array(post_freq)
          if max(con_freq):
            sentence += input[i] + pp[con_freq.argmax()] + " "
            total_freq *= max(con_freq)
          else:
            flag = 0
            break
      else:
        flag = 0
        break
        
    if flag:
      sentence += input[-1]
      made.append(sentence)
      prob.append(total_freq)
    else:
      made.append("")
      prob.append(0)

In [ ]:
for i in range(len(made)):
  print(made[i], prob[i])

강의를 예쁘고 네트워크 연결 2.54685680172095e-18
강의를 예쁘고 연결 네트워크 4.2714807019006783e-19
강의와 네트워크를 예쁘고 연결 2.13510196502431e-21
강의와 네트워크 연결을 예쁘 8.985815297002227e-19
강의를 연결을 예쁘고 네트워크 1.9050951420144736e-20
강의를 연결 네트워크를 예쁘 7.385995777579992e-18
예쁘고 강의와 네트워크 연결 8.120424100449313e-19
예쁘고 강의를 연결 네트워크 2.848003250368647e-18
예쁘고 네트워크를 강의를 연결 1.3753898631401337e-20
예쁘고 네트워크 연결을 강의 2.0371742765251417e-19
예쁘고 연결을 강의와 네트워크 2.938069129895917e-22
예쁘고 연결 네트워크를 강의 7.648531618784623e-20
네트워크를 강의를 예쁘고 연결 1.4561555929527965e-21
네트워크를 강의를 연결을 예쁘 1.074356932694201e-20
네트워크를 예쁘고 강의를 연결 3.151740579945546e-20
네트워크를 예쁘고 연결을 강의 3.7810471049897932e-22
네트워크 연결을 강의를 예쁘 6.415942852061109e-19
네트워크 연결을 예쁘고 강의 4.45901559075413e-19
연결을 강의를 예쁘고 네트워크 1.400550430884514e-21
연결을 강의와 네트워크를 예쁘 1.1580878629817607e-21
연결을 예쁘고 강의와 네트워크 1.384649162745876e-21
연결을 예쁘고 네트워크를 강의 7.7761598752536975e-22
연결 네트워크를 강의를 예쁘 5.18653484296654e-19
연결 네트워크를 예쁘고 강의 3.773719715841933e-19
키는 킥과 의의 3.840818937367657e-18
키와 의의가 킥 3.626467189051376e-17
킥은 키와 의의 4

In [ ]:
made[np.array(prob).argmax()]

'키와 의의가 킥'

In [ ]:
model[['예쁘']]

FreqDist({'ㄴ': 376, '게': 141, '고': 75, '다': 45, '어': 33, '지': 22, '다고': 12, '기': 12, '었': 10, '다는': 8, ...})

In [ ]:
model[['예쁘']].freq('다')

0.05555555555555555

In [ ]:
model[['다']]['네트워크']

5

In [ ]:
model[['강의']]['가']

178

In [ ]:
model[['강의']]['를']

1053

In [ ]:
model[['를']]['예쁘']

14

In [ ]:
model[['가']]['예쁘']

29

## 최종 코드

In [ ]:
def make(model, combs):
  made = ['']
  prob = [0]

  for t in combs:
    per = list(permutations(t))
    for k in range(len(per)):
      input = per[k]
      n = len(input)
      sentence = ""
      pp = ['은', '는', '이', '가', '을', '를', '와', '과', '다', '고']
      flag = 1
      total_freq = model.unigrams.freq(input[0])
      for i in range(n-1):
        if model[[input[i]]]:
          if model[[input[i]]][input[i+1]]:
            sentence += input[i] + " "
            total_freq *= model[[input[i]]].freq(input[i+1])
          else:
            # 조사 활용
            pre_freq = []
            post_freq = []
            for p in pp:
              pre_freq.append(model[[input[i]]].freq(p))
              post_freq.append(model[[p]].freq(input[i+1]))
            con_freq = np.array(pre_freq) * np.array(post_freq)
            if max(con_freq):
              sentence += input[i] + pp[con_freq.argmax()] + " "
              total_freq *= max(con_freq)
            else:
              flag = 0
              break
        else:
          flag = 0
          break
          
      if flag:
        sentence += input[-1]
        made.append(sentence)
        prob.append(total_freq)
      else:
        made.append("")
        prob.append(0)
  
  return made[np.array(prob).argmax()]

In [ ]:
make(model, [('키', '킥', '의의')])

'키와 의의가 킥'

In [ ]:
make(model, [('의', '의', '키킥')])

''

In [ ]:
make(model, [])

''